In [35]:
#import packages
import scipy.linalg as scilin
import numpy as np
import json
import pandas as pd


In [36]:
#Functions Section

def EigenPrint_TwoMat(A,B):
    C = np.matmul(np.linalg.inv(B),A)
    eigen_vals,eigen_vects = scilin.eig(C)
    id_list = eigen_vals.argsort()[::-1]
    eigen_vals = eigen_vals[id_list]
    eigen_vects = eigen_vects[:,id_list]
    for i in range(len(eigen_vals)):
        print(f'Eigen Value: {eigen_vals[i]}')
    for j in range(len(eigen_vects)):
        print(f'Eigenvector: {eigen_vects[j]}')

In [37]:
#Read in the data from the JSON file
filename_load = "0000.json"
Output_File_Long = 'Eigen_Long_Mode.txt'
J_string = open(filename_load).read()
J_vals = json.loads(J_string)

#################  Aircraft Properties  ##################################

#Get the moment information of aircraft
I_xx = J_vals['aircraft']['Ixx[slugs*ft^2]']
I_yy = J_vals['aircraft']['Iyy[slugs*ft^2]']
I_zz = J_vals['aircraft']['Izz[slugs*ft^2]']
I_xy = J_vals['aircraft']['Ixy[slugs*ft^2]']
I_xz = J_vals['aircraft']['Ixz[slugs*ft^2]']

#Name
Plane_name = J_vals['aircraft']['name']

#Wing Information
Wing_Area = J_vals['aircraft']['wing_area[ft^2]']
Wing_Span = J_vals['aircraft']['wing_span[ft]']
Wing_MeanChord = Wing_Area/Wing_Span

#aircraft weight
Weight = J_vals['aircraft']['weight[lbf]']

#Launch Energy
Launch_Energy = J_vals['aircraft']['launch_kinetic_energy[ft-lbf]']

#################  Air Properties  ########################################

#density
Air_Density = J_vals['analysis']['density[slugs/ft^3]']

#################  Aerodynamic Properties  #################################

#CL values
CL_0 = J_vals['aerodynamics']['CL']['0']
CL_alpha = J_vals['aerodynamics']['CL']['alpha']
CL_qbar = J_vals['aerodynamics']['CL']['qbar']
CL_alpha_hat = J_vals['aerodynamics']['CL']['alpha_hat']

#CY values
CY_beta = J_vals['aerodynamics']['CY']['beta']
CY_pbar = J_vals['aerodynamics']['CY']['pbar']
CY_rbar = J_vals['aerodynamics']['CY']['rbar']

#CD values
CD_L0 = J_vals['aerodynamics']['CD']['L0']
CD_L1 = J_vals['aerodynamics']['CD']['L']
CD_L2 = J_vals['aerodynamics']['CD']['L2']
CD_0 = CD_L0 + CD_L1*CL_0+CD_L2*CL_0**2
CD_alpha = CD_L1*CL_alpha + 2*CD_L2*CL_0*CL_alpha
CD_qbar = J_vals['aerodynamics']['CD']['qbar']

#Cl Values
Cl_beta = J_vals['aerodynamics']['Cl']['beta']
Cl_pbar = J_vals['aerodynamics']['Cl']['pbar']
Cl_rbar = J_vals['aerodynamics']['Cl']['rbar']

#Cm Values
Cm_0 = J_vals['aerodynamics']['Cm']['0']
Cm_alpha = J_vals['aerodynamics']['Cm']['alpha']
Cm_qbar = J_vals['aerodynamics']['Cm']['qbar']
Cm_alpha_hat = J_vals['aerodynamics']['Cm']['alpha_hat']

#Cn values
Cn_beta = J_vals['aerodynamics']['Cn']['beta']
Cn_pbar = J_vals['aerodynamics']['Cn']['pbar']
Cn_rbar = J_vals['aerodynamics']['Cn']['rbar']

#################  Solved Variables ############################################

#Gravity at location
g = 32.17 #gravity at logan elevation used for design

#Solve for the inital velocity given the launch conditions
#V_0 = np.sqrt(2*Launch_Energy*g/Weight) #Need to check if this is correct, don't do this, but we can do this later

#Get V_0 from the CL
V_0 = np.sqrt((Weight)/(0.5*Wing_Area*Air_Density*CL_0))



In [38]:
#Solving for all the Base Components found in eqs. 10.70-10.76 to then put into the matrices in 10.77 and 10.78, all variable must be unitless

#10.70 -10.72 not required because we don't know these variables and this is a glider

#10.73
R_gx = (g*Wing_MeanChord)/(2*V_0**2) 
R_gy = (g*Wing_Span)/(2*V_0**2) 
#10.74
R_rhox = (4*Weight/g)/(Air_Density*Wing_Area*Wing_MeanChord)
R_rhoy = (4*Weight/g)/(Air_Density*Wing_Area*Wing_Span)
#10.75 
R_xx = (8*I_xx)/(Air_Density*Wing_Area*Wing_Span**3)
R_yy = (8*I_yy)/(Air_Density*Wing_Area*Wing_MeanChord**3)
R_zz = (8*I_zz)/(Air_Density*Wing_Area*Wing_Span**3)
R_xz = (8*I_xz)/(Air_Density*Wing_Area*Wing_Span**3)
#10.76
#Not required since there is no thrust given this aircraft is a glider

In [39]:
#Generating the Matrices, going for the nondimensional form, Longitudinal Equations  
#10.77 B mat is the LHS matrix, A mat is the RHS matrix
A_mat_long = np.zeros([6,6])
B_mat_long = np.identity(6)

#Variables that could be changed, but assumed to be zero
CD_mu = 0
CL_mu_hat = 0
Cm_mu_hat = 0
CD_alpha_hat = 0
alpha_deg = 0
alpha_rad = alpha_deg*np.pi/180
#Fill in the A Matrix given what is known
    #Single Row
A_mat_long[0,0] = -2*CD_0 #+ CT_V*np.cos(alpha_rad)
A_mat_long[0,1] = CL_0-CD_alpha
A_mat_long[0,2] = -CD_qbar
A_mat_long[0,5] = -R_rhox*R_gx*np.cos(alpha_rad)
    #Second Row
A_mat_long[1,0] = -2*CL_0 #+ CT_V*np.sin(alpha_rad)
A_mat_long[1,1] = -CL_alpha-CD_0  #Variable is not correct currently, CD_0 is not correct
A_mat_long[1,2] = -CL_qbar+R_rhox
A_mat_long[1,5] = -R_rhox*R_gx*np.sin(alpha_rad)
    #Third Row
A_mat_long[2,0] = 2*Cm_0
A_mat_long[2,1] = Cm_alpha
A_mat_long[2,2] = Cm_qbar
    #Fourth Row
A_mat_long[3,0] = np.cos(alpha_rad)
A_mat_long[3,1] = np.sin(alpha_rad)
A_mat_long[3,5] = -np.sin(alpha_rad)
    #Fifth Row
A_mat_long[4,0] = -np.sin(alpha_rad)
A_mat_long[4,1] = np.cos(alpha_rad)
A_mat_long[4,5] = -np.cos(alpha_rad)
    #Sixth Row
A_mat_long[5,2] = 1

#Fill in the B Matrix given what is known
B_mat_long[0,0] = R_rhox+CD_mu
B_mat_long[0,1] = CD_alpha_hat
B_mat_long[1,0] = CL_mu_hat
B_mat_long[1,1] = R_rhox+CL_alpha_hat
B_mat_long[2,0] = -Cm_mu_hat
B_mat_long[2,1] = -Cm_alpha_hat
B_mat_long[2,2] = R_yy

#Create a New matrix using the A and B matrix to do the Eigen value solve

C_mat_long = np.matmul(np.linalg.inv(B_mat_long),A_mat_long)

pd.DataFrame(C_mat_long).head(6)


,0,1,2,3,4,5
0,-0.003742,0.030575,-0.009481,0.0,0.0,-0.047976
1,-0.090715,-0.397419,0.775011,0.0,0.0,0.000000
2,0.002286,-0.007711,-0.156836,0.0,0.0,0.000000
3,1.000000,0.000000,0.000000,0.0,0.0,0.000000
4,0.000000,1.000000,0.000000,0.0,0.0,-1.000000
5,0.000000,0.000000,1.000000,0.0,0.0,0.000000


In [40]:
#Getting the actual Eigenvalues and Eigenvectors

eigen_vals,eigen_vects = scilin.eig(C_mat_long)


# Print eigenvalues and eigenvectors
print("Eigenvalues:")
for i, val in enumerate(eigen_vals):
    print(f"Eigenvalue {i + 1}: {val.real:.4f} + {val.imag:.4f}j")

print("\nEigenvectors:")
for i in range(2,len(eigen_vects)):
    print(f"Eigenvector {i + 1}:")
    for j in range(len(eigen_vects)):
        print(f"   {eigen_vects[j, i].real:.4f} + {eigen_vects[j, i].imag:.4f}j")



Eigenvalues:
Eigenvalue 1: 0.0000 + 0.0000j
Eigenvalue 2: 0.0000 + 0.0000j
Eigenvalue 3: -0.3577 + 0.0000j
Eigenvalue 4: -0.1981 + 0.0000j
Eigenvalue 5: -0.0011 + 0.0330j
Eigenvalue 6: -0.0011 + -0.0330j

Eigenvectors:
Eigenvector 3:
   0.0306 + 0.0000j
   -0.3051 + 0.0000j
   -0.0121 + 0.0000j
   -0.0856 + 0.0000j
   0.9473 + 0.0000j
   0.0337 + 0.0000j
Eigenvector 4:
   -0.0381 + 0.0000j
   0.0935 + 0.0000j
   0.0196 + 0.0000j
   0.1926 + 0.0000j
   -0.9709 + 0.0000j
   -0.0988 + 0.0000j
Eigenvector 5:
   0.0009 + -0.0271j
   -0.0000 + 0.0050j
   -0.0001 + -0.0006j
   -0.8233 + 0.0000j
   0.0060 + -0.5666j
   -0.0187 + 0.0042j
Eigenvector 6:
   0.0009 + 0.0271j
   -0.0000 + -0.0050j
   -0.0001 + 0.0006j
   -0.8233 + -0.0000j
   0.0060 + 0.5666j
   -0.0187 + -0.0042j


In [41]:
#Get the amplitude and phase of each component as a numpy array from the eigenvectors

print(eigen_vects.shape)
amplitude = np.zeros_like(eigen_vects)
phase = np.zeros_like(eigen_vects)
#loop through columns
for j in range(eigen_vects.shape[0]):
     #loop through rows
     for i in range(eigen_vects.shape[1]):
          amplitude[i,j] = np.sqrt(eigen_vects[i,j].real**2 + eigen_vects[i,j].imag**2)
          phase[i,j] = np.arctan2(eigen_vects[i,j].imag,eigen_vects[i,j].real)*(180/np.pi)


pd.DataFrame(phase).head(6)



(6, 6)


,0,1,2,3,4,5
0,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,180.0+ 0.0j,-88.084314+ 0.000000j,88.084314+ 0.000000j
1,0.0+0.0j,0.0+0.0j,180.0+ 0.0j,0.0+ 0.0j,90.239451+ 0.000000j,-90.239451+ 0.000000j
2,0.0+0.0j,0.0+0.0j,180.0+ 0.0j,0.0+ 0.0j,-100.682013+ 0.000000j,100.682013+ 0.000000j
3,0.0+0.0j,0.0+0.0j,180.0+ 0.0j,0.0+ 0.0j,180.000000+ 0.000000j,-180.000000+ 0.000000j
4,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,180.0+ 0.0j,-89.396576+ 0.000000j,89.396576+ 0.000000j
5,0.0+0.0j,0.0+0.0j,0.0+ 0.0j,180.0+ 0.0j,167.402301+ 0.000000j,-167.402301+ 0.000000j


In [42]:
#Going through the damping rate

variable_symbols_lat = ['Δβ', 'Δp', 'Δr', 'Δξ_y', 'ΔΘ', 'Δφ']
variable_symbols_long = ['Δμ', 'Δα', 'Δq_bar', 'Δξ_x', 'Δξ_z', 'ΔΘ']


for z in range(len(eigen_vals)):
    i = eigen_vals[z]
    print(f'-------------------------------------------------------------------------------') 
    print(f'Dimensionless Eigen Value: {i.real:8.6f}+{i.imag:12.8f}j')
    sigma = -i.real
    #Rigid Body mode
    if i.real == 0:
        print('\t Rigid Body Mode: Eigen Value: 0 ')
        print('\t No analysis required currently\n')
    #Convergent Modes Sigma > 0
    if sigma > 0:
        #Damping rate
        Damp_rate = sigma*2*V_0/Wing_MeanChord
        #99 Damping Time
        Damp_time_99 = np.log(0.01)/-Damp_rate
        #Damped natural frequency and Period
        if i.imag != 0:
            W_d = abs(i.imag)*2*V_0/Wing_MeanChord
            Period = (2*np.pi)/W_d
        
        print(f'\t Damping Rate [1/s]: {Damp_rate:12.6f}')
        print(f'\t 99% Damping Time [s]: {Damp_time_99:12.6f}')
        if i.imag != 0:
            print(f'\t Damped Nat Freq: {W_d:12.6f}')
            print(f'\t Period: {Period:12.6f}')
        print('')
    #Divergent Modes Sigma < 0 
    if sigma < 0:
    #Damping rate
        Damp_rate = sigma*2*V_0/Wing_MeanChord
        #99 Damping Time
        Double_time = np.log(2.00)/-Damp_rate
        #Damped natural frequency and Period
        if i.imag != 0:
            W_d = abs(i.imag)*2*V_0/Wing_MeanChord
            Period = (2*np.pi)/W_d     

        print(f'\t Damping Rate [1/s]: {Damp_rate:12.6f}')
        print(f'\t Doubling Time [s]: {Double_time:12.6f}')
        if i.imag != 0:
            print(f'\t Damped Nat Freq: {W_d:12.6f}')
            print(f'\t Period: {Period:12.6f}')
        print('')  

    #Print the Eigen Vector
    print(f'-------------------------------------------------------------------------------') 
    print(f'{"Variable":<15} {"Real Part":<15} {"Imaginary Part":<20} {"Amplitude":<15} {"Phase":<15}')
    eigen_vects = np.asarray(eigen_vects)
    for j in range(6):
        real = eigen_vects[j,z].real
        imag = eigen_vects[j,z].imag
        amp = amplitude[j,z].real
        phase_deg = phase[j,z].real
        symbol = variable_symbols_long[j]
        print(f'{symbol:<15} {real:<15.6f} {imag:<20.6f} {amp:<15.6f} {phase_deg:<15.6f}')
    print(f'-------------------------------------------------------------------------------') 

            

-------------------------------------------------------------------------------
Dimensionless Eigen Value: 0.000000+  0.00000000j
	 Rigid Body Mode: Eigen Value: 0 
	 No analysis required currently

-------------------------------------------------------------------------------
Variable        Real Part       Imaginary Part       Amplitude       Phase          
Δμ              0.000000        0.000000             0.000000        0.000000       
Δα              0.000000        0.000000             0.000000        0.000000       
Δq_bar          0.000000        0.000000             0.000000        0.000000       
Δξ_x            1.000000        0.000000             1.000000        0.000000       
Δξ_z            0.000000        0.000000             0.000000        0.000000       
ΔΘ              0.000000        0.000000             0.000000        0.000000       
-------------------------------------------------------------------------------
----------------------------------------------

In [43]:
#Make this easy to write to a file so I can see it later

def write_info_file(output_file, eigen_vals, eigen_vects, amplitude, phase, V_0, Wing_MeanChord):
    # Define the variable symbols list manually
    variable_symbols_long = ['Δμ', 'Δα', 'Δq_bar', 'Δξ_x', 'Δξ_z', 'ΔΘ']

    # Open the output file for writing with UTF-8 encoding
    with open(output_file, 'w', encoding='utf-8') as file:
        for z in range(len(eigen_vals)):
            i = eigen_vals[z]
            file.write('-------------------------------------------------------------------------------\n') 
            file.write(f'Dimensionless Eigen Value: {i.real:8.6f}+{i.imag:12.8f}j\n')
            sigma = -i.real
            # Rigid Body mode
            if i.real == 0:
                file.write('\t Rigid Body Mode: Eigen Value: 0 \n')
                file.write('\t No analysis required currently\n\n')
            # Convergent Modes Sigma > 0
            if sigma > 0:
                # Damping rate
                Damp_rate = sigma*2*V_0/Wing_MeanChord
                # 99 Damping Time
                Damp_time_99 = np.log(0.01)/-Damp_rate
                # Damped natural frequency and Period
                if i.imag != 0:
                    W_d = abs(i.imag)*2*V_0/Wing_MeanChord
                    Period = (2*np.pi)/W_d
                
                file.write(f'\t Damping Rate [1/s]: {Damp_rate:12.6f}\n')
                file.write(f'\t 99% Damping Time [s]: {Damp_time_99:12.6f}\n')
                if i.imag != 0:
                    file.write(f'\t Damped Nat Freq: {W_d:12.6f}\n')
                    file.write(f'\t Period: {Period:12.6f}\n\n')
            # Divergent Modes Sigma < 0 
            if sigma < 0:
                # Damping rate
                Damp_rate = sigma*2*V_0/Wing_MeanChord
                # 99 Damping Time
                Double_time = np.log(2.00)/-Damp_rate
                # Damped natural frequency and Period
                if i.imag != 0:
                    W_d = abs(i.imag)*2*V_0/Wing_MeanChord
                    Period = (2*np.pi)/W_d     

                file.write(f'\t Damping Rate [1/s]: {Damp_rate:12.6f}\n')
                file.write(f'\t Doubling Time [s]: {Double_time:12.6f}\n')
                if i.imag != 0:
                    file.write(f'\t Damped Nat Freq: {W_d:12.6f}\n')
                    file.write(f'\t Period: {Period:12.6f}\n\n')

            # Print the Eigen Vector
            file.write('-------------------------------------------------------------------------------\n') 
            file.write(f'{"Variable":<15} {"Real Part":<15} {"Imaginary Part":<20} {"Amplitude":<15} {"Phase":<15}\n')
            eigen_vects = np.asarray(eigen_vects)
            for j in range(6):
                real = eigen_vects[j,z].real
                imag = eigen_vects[j,z].imag
                amp = amplitude[j,z].real
                phase_deg = phase[j,z].real
                symbol = variable_symbols_long[j]
                file.write(f'{symbol:<15} {real:<15.6f} {imag:<20.6f} {amp:<15.6f} {phase_deg:<15.6f}\n')
            file.write('-------------------------------------------------------------------------------\n')




write_info_file(Output_File_Long,eigen_vals,eigen_vects,amplitude,phase,V_0,Wing_MeanChord)